In [67]:
!pip install transformers openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 871 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 8.9 MB/s 
  Created wheel for openai: filename=openai-0.19.0-py3-none-any.whl size=53535 sha256=427d033c071e2afca976cc48c4b570f9641fd799b9df5b30a3248b8eca0ee70a
  Stored in directory: /root/.cache/pip/wheels/94/b5/c0/928013bd6418b23b9c5d89fb24cdeb1faae899c11377d69609
Successfully built openai


In [2]:
import os 
import sys
import csv
import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/frame')

import csv
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import torch
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from collections import defaultdict
import numpy as np

Mounted at /content/drive


In [441]:
train_arguments = []
train_frames = []
train_topics = []
with open('/content/drive/MyDrive/LT_project_data/Train.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        train_arguments.append(row[2])
        train_frames.append(row[-1])
        train_topics.append(row[4])

val_arguments = []
val_frames = []
val_topics = []
with open('/content/drive/MyDrive/LT_project_data/Validation.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        val_arguments.append(row[2])
        val_frames.append(row[-1])
        val_topics.append(row[4])

test_arguments = []
test_frames = []
test_topics = []
with open('/content/drive/MyDrive/LT_project_data/Test.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        test_arguments.append(row[2])
        test_frames.append(row[-1])
        test_topics.append(row[4])

In [442]:
# # REMOVE 'Other' and 'Irrelevant' classes

train = list(zip(train_arguments, train_frames,train_topics))
val = list(zip(val_arguments, val_frames,val_topics))
test = list(zip(test_arguments, test_frames,test_topics))

train = [s for s in train if s[1] not in ['Other']]
val = [s for s in val if s[1] not in ['Other']]
test = [s for s in test if s[1] not in ['Other']]

train_arguments = list(list(zip(*train))[0])
train_frames = list(list(zip(*train))[1])
train_topics = list(list(zip(*train))[2])

val_arguments = list(list(zip(*val))[0])
val_frames = list(list(zip(*val))[1])
val_topics = list(list(zip(*val))[2])

test_arguments = list(list(zip(*test))[0])
test_frames = list(list(zip(*test))[1])
test_topics = list(list(zip(*test))[2])

In [448]:
# # REMOVE IMPORTANT FEATURES PER TOPIC

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_arguments)
le = preprocessing.LabelEncoder()
le.fit(train_topics)
Y = le.transform(train_topics)

clf = SGDClassifier(loss='log')
clf.fit(X, Y)

labeldict = defaultdict(list)
for i in range(0, clf.coef_.shape[0]):
    top20_indices = np.argsort(clf.coef_[i])[-20:]
    for j in top20_indices:
      labeldict[le.classes_[i]].append(vectorizer.get_feature_names()[j])

train = list(zip(train_arguments, train_topics))
val = list(zip(val_arguments, val_topics))
test = list(zip(test_arguments, test_topics))

train_arguments = []
for i in train:
  sent = i[0].split()
  for word in sent:
    if word in labeldict[i[1]]:
      sent[sent.index(word)] = '[MASK]'
  train_arguments.append(' '.join(sent))

val_arguments = []
for i in val:
  sent = i[0].split()
  for word in sent:
    if word in labeldict[i[1]]:
      sent[sent.index(word)] = '[MASK]'
  val_arguments.append(' '.join(sent))

test_arguments = []
for i in test:
  sent = i[0].split()
  for word in sent:
    if word in labeldict[i[1]]:
      sent[sent.index(word)] = '[MASK]'
  test_arguments.append(' '.join(sent))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [449]:
# add one example of each class to the prompts

import random
prompts = []
uniqprompt = []
a = list(zip(train_arguments,train_frames))
random.shuffle(a)
for x,y in a:
  if len(x.split()) < 100:
    if y not in uniqprompt:
      # prompts.append('Argument: {0}\nClass: {1}'.format(x,y))
      prompts.append('{0} The frame of this argument is: {1}'.format(x,y))
      uniqprompt.append(y)

In [189]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
import os
import openai

openai.api_key = 'KEY HERE'


preds = []

prompt = ''
for i in prompts:
  prompt += i
  prompt += '\n'

for i in test_arguments:
  prompt_temp = prompt
  restart_sequence = '\n'
  try:

    # FEW-SHOT PROMPT COMPLETION
    prompt_temp = prompt_temp + '{0} The frame of this argument is:'.format(i)

    # ZERO-SHOT CLASSIFICATION
    # prompt_temp = '''
    # The following is a list of classes:\n\nCapacity and resources, Technology and innovation, Morality, International relations and reputation, Legality, constitutionality and jurisprudence, Historical, Economic, Health and Safety, Crime and punishment, Policy prescription and evaluation, Cultural identity, Political, Fairness and equality, Climate and environment, Public opinion, Quality of Life\n\nArgument: {0}\nClass:
    # '''.format(i)

    # FEW-SHOT CLASSIFICATION
    # prompt_temp = prompt_temp + '''
    # The following is a list of classes:\n\nCapacity and resources, Technology and innovation, Morality, International relations and reputation, Legality, constitutionality and jurisprudence, Historical, Economic, Health and Safety, Crime and punishment, Policy prescription and evaluation, Cultural identity, Political, Fairness and equality, Climate and environment, Public opinion, Quality of Life\n\nArgument: {0}\nClass:
    # '''.format(i)

    # For the following, change the engine to 'text-davinci-002' for acceptable results (be cautious: this is expensive)

    # ZERO-SHOT QA
    # prompt_temp = '''
    # Q: {0} What is the frame of this argument? \n A. Capacity and resources \n B. Technology and innovation \n C. Morality \n D. International relations and reputation \n E. Legality, constitutionality and jurisprudence \n F. Historical \n G. Economic \n H. Health and Safety \n I. Crime and punishment \n J. Policy prescription and evaluation \n K. Cultural identity \n L. Political \n M. Fairness and equality \n N. Climate and environment \n O. Public opinion \n P. Quality of Life \n A:
    # '''.format(i)

    # FEW-SHOT QA
    # prompt_temp = prompt_temp + '''
    # Q: {0} What is the frame of this argument? \n A. Capacity and resources \n B. Technology and innovation \n C. Morality \n D. International relations and reputation \n E. Legality, constitutionality and jurisprudence \n F. Historical \n G. Economic \n H. Health and Safety \n I. Crime and punishment \n J. Policy prescription and evaluation \n K. Cultural identity \n L. Political \n M. Fairness and equality \n N. Climate and environment \n O. Public opinion \n P. Quality of Life \n A:
    # '''.format(i)

    response = openai.Completion.create(
      engine="text-curie-001",
      prompt=prompt_temp,
      temperature=0,
      max_tokens=4,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
    )
    preds.append(response['choices'][0]['text'].strip())

  except:
    preds.append('Cultural identity')



In [488]:
# Levenshtein mapping

classes = list(set(test_frames))

from Levenshtein import distance as levenshtein_distance
real_preds = []
for i in preds:
  probs = []
  for j in classes:
    probs.append(levenshtein_distance(i,j))

  real_preds.append(classes[probs.index(min(probs))])

In [490]:
from sklearn.metrics import accuracy_score, f1_score
print(preds)
print(test_frames)

for x,y in zip(preds,test_frames):
  if x.lower().strip() == y.lower().strip():
    print(x)

print(f1_score(real_preds,test_frames, average='macro'))

['Ethics', 'History', 'Politics', 'Ethics', 'Social justice', 'Politics', 'Social policy', 'History', 'Politics', 'Politics', 'Economics', 'History', 'Politics', 'Politics', 'Political', 'History', 'Freedom and liberty', 'Social justice', 'Social', 'Politics', 'Social justice', 'Social dynamics', 'Politics', 'Social policy', 'History', 'Politics', 'Political', 'Ethics', 'Political', 'Political', 'Morality', 'Business', 'Ethics', 'Religion', 'Civil rights', 'Business', 'Religion', 'Ethics', 'Ethics', 'Religion', 'Business', 'Religion', 'Businesses and commerce', 'Religion', 'Religion', 'Religion', 'Religion', 'Economics', 'Economics', 'Business', 'Religion', 'Businesses and the', 'Civil rights', 'Religion', 'Morality', 'Religion', 'Religion', 'Religion', 'Morality', 'Religion', 'Religion', 'Ethics', 'Religion', 'Business', 'Morality', 'Morality', 'Sexual orientation', 'Ethics', 'Social', 'Law and justice', 'Law', 'Morality', 'Marriage', 'Marriage', 'Law and justice', 'Social', 'Law and 